<a href="https://colab.research.google.com/github/DQKinn/dartswPytorch/blob/main/vgg16_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [9]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.4914,0.4822, 0.4465),(0.2470,0.2435,0.2616))])
device = torch.device('cpu')

In [10]:
trainset = datasets.CIFAR10(root='./data',
                            train=True,
                            download=True,
                            transform=transform)
testset = datasets.CIFAR10(root='./data',
                            train=False,
                            download=True,
                            transform=transform)
train_loader = DataLoader(trainset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=1)
test_loader = DataLoader(testset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

Files already downloaded and verified
Files already downloaded and verified


In [20]:
VGG16=[64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M']
class VGGnet(nn.Module):
  def __init__ (self, in_channels= 3, num_classes = 10):
    super(VGGnet, self).__init__()
    self.in_channels = in_channels
    self.conv_layer = self.create_conv_layer(VGG16)
    self.fcs = nn.Sequential(
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,num_classes)
    )
  def forward(self,x):
    x= self.conv_layer(x)
    x= x.reshape(x.shape[0],-1)
    x= self.fcs(x)
    return x
  
  def create_conv_layer(self, architecture):
    layers=[]
    in_channels = self.in_channels

    for x in architecture:
      if type(x) == int:
        out_channels = x
        layers += [nn.Conv2d(in_channels=in_channels, out_channels = out_channels,
                             kernel_size=3, stride =1, padding =1),nn.BatchNorm2d(x),nn.ReLU()]
        in_channels = x
      elif x == 'M':
        layers += [nn.MaxPool2d(kernel_size = 2, stride = 2)]
      
    return nn.Sequential(*layers)

model = VGGnet(in_channels =3, num_classes = 10)


In [21]:
from torchsummary import summary
a= torch.randn(3,32,32).to(device)
model(a.unsqueeze(0)).to(device)
print(model)

VGGnet(
  (conv_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), paddin

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(params=model.parameters(), lr=2e-3, betas=(0.9, 0.999))

In [ ]:
import datetime

for epoch in range(20):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  # print('{} Epoch {}, Training loss , {}loss {}'.format(
  # datetime.datetime.now(), epoch,
  # running_loss / len(train_loader), loss))


In [25]:
PATH = './vgg.pt'
torch.save(model.state_dict(), PATH)

In [26]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print(100*correct/total)

77.68
